# A short demo on how to use the stlcg toolbox

### Remember, sequences are reversed!

In [1]:
%matplotlib inline
import torch
import numpy as np
from abc import ABC, abstractmethod
from src.stlcg import *
from src.stlviz import *
from src.util import *

import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


ModuleNotFoundError: No module named 'scripts'

In [ ]:
x_np = np.array([5, 4, 3, 2, 1, 0, 1, 2, 3, 4], dtype=np.float32).reshape([1, 10, 1])
w_np = np.array([0, 1, 2, 3, 4, 3, 3, 3, 2, 2], dtype=np.float32).reshape([1, 10, 1])
x = torch.tensor(x_np, requires_grad=False)
w = torch.tensor(w_np, requires_grad=False)
c = torch.tensor(4.0, dtype=torch.float, requires_grad=True)
d = torch.tensor(4.0, dtype=torch.float, requires_grad=True)

In [ ]:
GThan = GreaterThan(name="x", c=c)
print(GThan)
LThan = LessThan(name="w", c=d)
print(LThan)
Eq = Equal(name="x", c=d)
print(Eq)
#An = And(subformula1=LThan, subformula2=GThan)
An = ~LThan #LThan & GThan
print(An)
Alw = Always(subformula=An)
print(Alw)
Ev = Eventually(subformula=An)
print(Ev)
Unt = Until(subformula1=GThan, subformula2=Always(subformula=LThan))
print(Unt)

In [ ]:
make_stl_graph(GThan)

In [ ]:
make_stl_graph(LThan)

In [ ]:
make_stl_graph(Eq)

In [ ]:
make_stl_graph(Alw)

In [ ]:
make_stl_graph(Ev)

In [ ]:
make_stl_graph(Unt)

## Saving a graph

In [ ]:
dot = make_stl_graph(Unt)
save_graph(dot, "until_graph")

# Testing grad functionality

In [ ]:
device = torch.device("cpu")
learning_rate = 0.01
def print_learning_progress(var_dict, i, loss):
    vals = [i, loss]
    string = "iteration: %i -- loss: %.3f"
    for (k,v) in var_dict.items():
        string += " ---- %s:%.3f"
        vals.append(k)
        vals.append(v)
    print(string%tuple(vals))

In [ ]:
x_np = np.array([5, 4, 3, 2, 1, 0, 1, 2, 3, 4], dtype=np.float32).reshape([1, 10, 1])
w_np = np.array([2, 2, 2, 3, 4, 3, 3, 3, 2, 2], dtype=np.float32).reshape([1, 10, 1])
x = torch.tensor(x_np, requires_grad=False)
w = torch.tensor(w_np, requires_grad=False)
c = torch.tensor(6.0, dtype=torch.float, requires_grad=True)
d = torch.tensor(1.0, dtype=torch.float, requires_grad=True)
W = w + torch.randn([20,10,1], requires_grad=False)


In [ ]:
GThan = GreaterThan(name="w", c=c)
LThan = LessThan(name="w", c=d)

Unt = Until(subformula1=LThan, subformula2=Always(subformula=GThan))
Th = Then(subformula1=LThan, subformula2=GThan)
print(Th)
model = Th
var_dict = {"c":c, "d":d}


optimizer = torch.optim.Adam(var_dict.values(), lr=learning_rate)
scale = 5
make_stl_graph(model)

In [ ]:
for i in range(500):
    learning_rate = 0.01 - 0.01*i/500. + 0.000001
    scale = 5 - i/500.
    trace1 = model.subformula1(w, scale=scale)
    trace2 = model.subformula2(w, scale=scale)
    robustness = model.robustness(trace1, trace2, scale=scale)
#     print(robustness)
    loss = torch.abs(model.robustness(trace1, trace2, scale=scale)).mean()
    print_learning_progress(var_dict, i, loss)
    loss.backward()
    with torch.no_grad():
        c -= learning_rate * c.grad
        c.grad.zero_()
        d -= learning_rate * d.grad
        d.grad.zero_()